# Load data for Machine Learning and Deep Learning

This Notebook covers information about loading data specifically for ML and DL application

## Petastorm

Petastorm is an opensource data access library that enables directly loading data stored in Apache Parquet Format. This library enables single-node or distributed training and evaluation of deep learning model directly from datasets in Apache parquet format and dataset that are loaded as Apache Spark Data frame

Petastorm Spark convertor API simplifies data conversion from Spark to Tensorflow or Pytorch.

In [7]:
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName('Loading Data').getOrCreate()

In [8]:
from petastorm.spark import SparkDatasetConverter, make_spark_converter
spark.conf.set(SparkDatasetConverter.PARENT_CACHE_DIR_URL_CONF, 'file:///home/jovyan/cache')

In [5]:
from pyspakr

/home/jovyan/Machine_learning/pyspark
